# M2608.001300 Machine Learning<br> Assignment #5 Final Projects (Pytorch)


## Cap2TxT : An End-to-End Hybrid Neural Network for Captcha Image Text Sequence Recognition

<div style="text-align: right">
Changwoon Choi.<br>
SNU ECE.<br>
2014-17733<br>
</div>

For code simplication of this main.ipynb file, I added some python files for simple calculation or transformation at current directory.<br> (those codes are also submitted with this main.ipynb file with maintaining the directory hierarchy.)

In [1]:
import os
import random

import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.utils.data
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
from torch.nn import CTCLoss
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader,Dataset
from torch.autograd import Variable
from PIL import Image
import matplotlib.pyplot as plt

import dataset
import utils
import params

Load datasets


In [2]:
warnings.filterwarnings("ignore", category=DeprecationWarning)
alphabet='0123456789abcdefghijklmnopqrstuvwxyz'
NUMBER = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
ALPHABET = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
NONE = ['NONE'] # label for empty space
ALL_CHAR_SET = NUMBER + ALPHABET + NONE
ALL_CHAR_SET_LEN = len(ALL_CHAR_SET)
MAX_CAPTCHA = 7

def encode(a):
    onehot = [0]*ALL_CHAR_SET_LEN
    idx = ALL_CHAR_SET.index(a)
    onehot[idx] += 1
    return onehot

class Mydataset(Dataset):
    def __init__(self, img_path, label_path, is_train=True, transform=None):
        self.path = img_path
        self.label_path = label_path
        if is_train: 
            self.img = os.listdir(self.path)[:10000]
            self.labels = open(self.label_path, 'r').read().split('\n')[:-1][:10000]
        else: 
            self.img = os.listdir(self.path)[:1000]
            self.labels = open(self.label_path, 'r').read().split('\n')[:-1][:1000]
        
        self.transform = transform
        self.max_length = MAX_CAPTCHA
        
    def __getitem__(self, idx):
        img_path = self.img[idx]
        img = Image.open(f'{self.path}/{self.img[idx]}')
        img = img.convert('L')
        label = self.labels[idx]
        label_oh = []
        
        for i in range(self.max_length):
            if i < len(label):
                label_oh += encode(label[i])
            else:
                #label_oh += [0]*ALL_CHAR_SET_LEN
                label_oh += encode('NONE')
            
        if self.transform is not None:
            img = self.transform(img)

        return img, np.array(label_oh), label
    def __len__(self):
        return len(self.img)

transform = transforms.Compose([
    transforms.Resize([160, 60]),
    transforms.ToTensor()
])

random.seed(1234)
np.random.seed(1234)
torch.manual_seed(1234)


In [3]:
gPath = './'
train_ds = Mydataset(gPath+'Data/train/', gPath+'Data/train.txt',transform=transform)
test_ds = Mydataset(gPath+'Data/test/', gPath+'Data/test.txt',False, transform)
train_dl = DataLoader(train_ds, batch_size=64, num_workers=4)
test_dl = DataLoader(test_ds, batch_size=1, num_workers=4)
my_train_loader=torch.utils.data.DataLoader(dataset.lmdbDataset('Data/train_prepared'), batch_size=32,shuffle=True, sampler=None, num_workers=4, collate_fn=dataset.alignCollate(imgH=64,imgW=160, keep_ratio=False))

In [ ]:
#prepare dataset (the python files below should be executed only once.)

In [14]:
!python misc.py

In [15]:
!python misc_2.py

In [17]:
!python misc_3.py

In [10]:
!python create_dataset.py --out Data/train_prepared --folder Data/train

create_dataset.py:14: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  imageBuf = np.fromstring(imageBin, dtype=np.uint8)


In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Problem 1: Design LSTM model for captcha image recognition. (10 points)

In [14]:
class ONE_LAYER_LSTM(nn.Module):
    def __init__(self, dim_in, dim_hidden, dim_out):
        super(ONE_LAYER_LSTM,self).__init__()
        self.lstm = nn.LSTM(dim_in, dim_hidden, bidirectional=True)
        self.fc_out = nn.Linear(2*dim_hidden, dim_out)
    def forward(self, features):
        tmp, _ = self.lstm(features)
        T,b,h = tmp.size()
        tmp = tmp.view(T*b,h)
        output = self.fc_out(tmp)
        output = output.view(T,b,-1)
        return output

class LSTM(nn.Module):
    def __init__(self, cnn_dim, hidden_size, classnum, num_layers=2):
        super(LSTM, self).__init__()
        self.lstmnet = nn.Sequential(ONE_LAYER_LSTM(cnn_dim, hidden_size, hidden_size),
                                    ONE_LAYER_LSTM(hidden_size,hidden_size, classnum))
        
    def forward(self, features):
        output = self.lstmnet(features)
        output = F.log_softmax(output,dim=2)
        
        return output




Problem 2: 

*   1.Connect CNN model to the designed LSTM model.
*   2.Replace ResNet to your own CNN model from Assignment3.


          


In [38]:
# trial1
'''my betternet from AS3'''

class Inception(nn.Module):
    def __init__(self, in_planes, n1x1, n3x3red, n3x3, n5x5red, n5x5, pool_planes):
        super(Inception, self).__init__()
        # 1x1 conv branch
        self.b1 = nn.Sequential(
            nn.Conv2d(in_planes, n1x1, kernel_size=1),
            nn.BatchNorm2d(n1x1),
            nn.ReLU(True),
        )
        self.b2 = nn.Sequential(
            nn.Conv2d(in_planes, n3x3red, kernel_size=1),
            nn.BatchNorm2d(n3x3red),
            nn.ReLU(True),
            nn.Conv2d(n3x3red, n3x3, kernel_size=3, padding=1),
            nn.BatchNorm2d(n3x3),
            nn.ReLU(True),
        )
        self.b3 = nn.Sequential(
            nn.Conv2d(in_planes, n5x5red, kernel_size=1),
            nn.BatchNorm2d(n5x5red),
            nn.ReLU(True),
            nn.Conv2d(n5x5red, n5x5, kernel_size=5, padding=2),
            nn.BatchNorm2d(n5x5),
            nn.ReLU(True),
            nn.Conv2d(n5x5, n5x5, kernel_size=5, padding=2),
            nn.BatchNorm2d(n5x5),
            nn.ReLU(True),
        )
        self.b4 = nn.Sequential(
            nn.MaxPool2d(3, stride=1, padding=1),
            nn.Conv2d(in_planes, pool_planes, kernel_size=1),
            nn.BatchNorm2d(pool_planes),
            nn.ReLU(True),
        )
    def forward(self, x):
        y1 = self.b1(x)
        y2 = self.b2(x)
        y3 = self.b3(x)
        y4 = self.b4(x)
        return torch.cat([y1,y2,y3,y4], 1)
    
class BetterNet_1(nn.Module):
    def __init__(self):
        super(BetterNet_1, self).__init__()        
        self.block1 = nn.Sequential(
            nn.Conv2d(in_channels=1,out_channels=64,kernel_size=7,stride=1,padding=3),
            nn.ReLU(True),
            nn.MaxPool2d(3,2,1)
        )
        self.block2 = nn.Sequential(
            nn.Conv2d(in_channels=64,out_channels=64,kernel_size=1,stride=1),
            nn.ReLU(True),
            nn.Conv2d(in_channels=64,out_channels=192,kernel_size=3,stride=1,padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(3,2,1)
        )
        self.incep_block1 = nn.Sequential(
            Inception(192,64,96,128,16,32,32),
            Inception(256,128,128,192,32,96,64),
            nn.MaxPool2d(3,2,1)
        )        
        self.incep_block2 = nn.Sequential(
            Inception(480,192,96,208,16,48,64),
            Inception(512,160,112,224,24,64,64),
            Inception(512,128,128,256,24,64,64),
            Inception(512,112,144,288,32,64,64),
            Inception(528,256,160,320,32,128,128),
            nn.MaxPool2d(3,2,1)
        )        
        self.incep_block3 = nn.Sequential(
            Inception(832,256,160,320,32,128,128),
            Inception(832,384,192,384,48,128,128),
            nn.AvgPool2d(kernel_size=4,stride=1,padding=0)
        )        
        self.fc_block = nn.Sequential(
            nn.Linear(1024,512),
            nn.ReLU(True),
            nn.Linear(512,ALL_CHAR_SET_LEN*MAX_CAPTCHA)
           # nn.ReLU(True),
           # nn.Linear(64,10)
        )        
    def forward(self, x):
        x=self.block1(x)
        x=self.block2(x)
        x=self.incep_block1(x)
        x=self.incep_block2(x)
        x=self.incep_block3(x)
        #print('cnn 5:',end='')
        #print(x.shape)
        #x = x.view(-1,1024)
        #x=self.fc_block(x)
        out=x
        return out


In [15]:
# trial 2
'''my betternet from AS3'''
class BetterNet_2(nn.Module):
    def __init__(self):
        super(BetterNet_2, self).__init__()
        
        self.cnn1=nn.Sequential()
        self.cnn1.add_module('conv1', nn.Conv2d(1,64,3,1,1))
        self.cnn1.add_module('relu1', nn.ReLU(True))
        self.cnn1.add_module('pooling1', nn.MaxPool2d(2,2))
        
        self.cnn2=nn.Sequential()
        self.cnn2.add_module('conv2', nn.Conv2d(64,128,3,1,1))
        self.cnn2.add_module('relu2', nn.ReLU(True))
        self.cnn2.add_module('pooling2', nn.MaxPool2d(2,2))
        
        self.cnn3=nn.Sequential()
        self.cnn3.add_module('conv3', nn.Conv2d(128,256,3,1,1))
        self.cnn3.add_module('batchnorm', nn.BatchNorm2d(256))
        self.cnn3.add_module('relu3', nn.ReLU(True))
               
        self.cnn4=nn.Sequential()
        self.cnn4.add_module('conv4', nn.Conv2d(256,256,3,1,1))
        self.cnn4.add_module('relu4', nn.ReLU(True))
        self.cnn4.add_module('pooling4', nn.MaxPool2d((2,2),(2,1),(0,1)))
        
        self.cnn5=nn.Sequential()
        self.cnn5.add_module('conv5', nn.Conv2d(256,512,3,1,1))
        self.cnn5.add_module('batchnorm', nn.BatchNorm2d(512))
        self.cnn5.add_module('relu5', nn.ReLU(True))

        self.cnn6=nn.Sequential()
        self.cnn6.add_module('conv6', nn.Conv2d(512,512,3,1,1))
        self.cnn6.add_module('relu6', nn.ReLU(True))
        self.cnn6.add_module('pooling6', nn.MaxPool2d((2,2),(2,1),(0,1)))
        
        self.cnn7=nn.Sequential()
        self.cnn7.add_module('conv7', nn.Conv2d(512,512,2,1,0))
        self.cnn7.add_module('batchnorm', nn.BatchNorm2d(512))
        self.cnn7.add_module('relu7', nn.ReLU(True))
        self.cnn7.add_module('pooling7', nn.AvgPool2d(kernel_size=3,stride=1))
        
    def forward(self,x):
        out=self.cnn1(x)
        out=self.cnn2(out)
        out=self.cnn3(out)
        out=self.cnn4(out)
        out=self.cnn5(out)
        out=self.cnn6(out)
        out=self.cnn7(out)
        return out
        
        


In [16]:
def weight_normalize(net):
    a=net.__class__.__name__
    if a.find('Conv') != -1:
        net.weight.data.normal_(0,0.02)
    elif a.find('BatchNorm') !=-1:
        net.weight.data.normal_(1,0.02)
        net.bias.data.fill_(0)

In [41]:
# my CNN-RNN fusioned model, Cap2TxT Network
# trial 1,2
vocab_size = len(alphabet)+1

class Cap2TxT(nn.Module):
    def __init__(self, hidden_size, class_num, num_layers=2):
        super(Cap2TxT, self).__init__()
        #self.cnn = BetterNet_2()
        self.cnn = BetterNet_1()
        #self.rnn = LSTM(512, hidden_size, class_num,num_layers)
        self.rnn = LSTM(1024, hidden_size, class_num, num_layers)
    def forward(self, x):
        
        features = self.cnn(x)
        #print(features.size())
        b,c,h,w = features.size()
        features = features.squeeze(2)
        features = features.permute(2,0,1)
        output = self.rnn(features)
        return output


hidden_size=256
batch_size = 32
lr = 0.0001

image = torch.FloatTensor(batch_size, 3, 64, 64)
text = torch.LongTensor(batch_size * 5)
length = torch.LongTensor(batch_size)

net = Cap2TxT(hidden_size,vocab_size,2)
net.apply(weight_normalize)


converter = utils.strLabelConverter(alphabet)
loss_func = CTCLoss()
loss_avg=utils.averager()

loss_func = loss_func.cuda()
image=image.cuda()
text=text.cuda()
net=net.cuda()

image=Variable(image)
text=Variable(text)
length=Variable(length)

cap2txt_optim = optim.RMSprop(net.parameters(),lr)
#cap2txt_optim = optim.Adam(net.parameters(),lr,betas=(0.5,0.999))



Problem3: Find hyper-parameters.


In [42]:
"""TRAINING"""

SAVE_PATH = './ckpt/'
display_interval = 1
save_interval=3
max_epoch = 250


for epoch in range(max_epoch):
    train_it = iter(my_train_loader)
    it=0
    while it<len(my_train_loader):
        for param in net.parameters():
            param.requires_grad=True
        net.train()
        chunk = train_it.next()
        img, label = chunk
        utils.loadData(image,img)
        txt, lth = converter.encode(label)
        utils.loadData(text,txt)
        utils.loadData(length,lth)
        
        cap2txt_optim.zero_grad()
        predict = net(image)
        predict_size = Variable(torch.LongTensor([predict.size(0)] * img.size(0)))
        
        loss = loss_func(predict,text,predict_size,length)/img.size(0)
        loss.backward()
        cap2txt_optim.step()
        
        loss_avg.add(loss)
        it+=1
        
    if epoch%display_interval==0:
        print('epoch : %d, Loss : %f' %(epoch,loss_avg.val()))
        loss_avg.reset()
    if epoch%save_interval==0:
        print('saved')
        #torch.save(net.state_dict(),SAVE_PATH="Cap2TxT_"+str(epoch)+'pth')
        torch.save(net.state_dict(),SAVE_PATH+'Cap2TxT_GoogLe_'+str(epoch)+'.pth')

print('Finished Training')
torch.save(net.state_dict(),SAVE_PATH+'Cap2TxT'+'_final.pth')
print('Saved Model')

epoch : 0, Loss : 0.122119
saved
epoch : 1, Loss : 0.114869
epoch : 2, Loss : 0.097594
epoch : 3, Loss : 0.080764
saved
epoch : 4, Loss : 0.063179
epoch : 5, Loss : 0.047502
epoch : 6, Loss : 0.036180
saved
epoch : 7, Loss : 0.029120
epoch : 8, Loss : 0.022502
epoch : 9, Loss : 0.018161
saved
epoch : 10, Loss : 0.015251
epoch : 11, Loss : 0.012913
epoch : 12, Loss : 0.010618
saved
epoch : 13, Loss : 0.009241
epoch : 14, Loss : 0.007873
epoch : 15, Loss : 0.007073
saved
epoch : 16, Loss : 0.006063
epoch : 17, Loss : 0.005284
epoch : 18, Loss : 0.005042
saved
epoch : 19, Loss : 0.006001
epoch : 20, Loss : 0.003667
epoch : 21, Loss : 0.004812
saved
epoch : 22, Loss : 0.004109
epoch : 23, Loss : 0.003277
epoch : 24, Loss : 0.002493
saved
epoch : 25, Loss : 0.002988
epoch : 26, Loss : 0.002113
epoch : 27, Loss : 0.002185
saved
epoch : 28, Loss : 0.002145
epoch : 29, Loss : 0.001959
epoch : 30, Loss : 0.001460
saved
epoch : 31, Loss : 0.001873
epoch : 32, Loss : 0.001267
epoch : 33, Loss : 0

In [37]:
'''Test'''

def get_char_count(arg1):
    c0 = ALL_CHAR_SET[np.argmax(arg1[0:ALL_CHAR_SET_LEN])]
    c1 = ALL_CHAR_SET[np.argmax(arg1[ALL_CHAR_SET_LEN:ALL_CHAR_SET_LEN*2])]
    c2 = ALL_CHAR_SET[np.argmax(arg1[ALL_CHAR_SET_LEN*2:ALL_CHAR_SET_LEN*3])]
    c3 = ALL_CHAR_SET[np.argmax(arg1[ALL_CHAR_SET_LEN*3:ALL_CHAR_SET_LEN*4])]
    c4 = ALL_CHAR_SET[np.argmax(arg1[ALL_CHAR_SET_LEN*4:ALL_CHAR_SET_LEN*5])]
    c5 = ALL_CHAR_SET[np.argmax(arg1[ALL_CHAR_SET_LEN*5:ALL_CHAR_SET_LEN*6])]
    c6 = ALL_CHAR_SET[np.argmax(arg1[ALL_CHAR_SET_LEN*6:ALL_CHAR_SET_LEN*7])]
    return c0, c1, c2,c3, c4, c5, c6 

def oh_encoding(a):
    label_oh = []
    for i in range(7):
        if i<len(a):
            label_oh+=encode(a[i])
        else:
            label_oh+=encode('NONE')
    return label_oh



#model_path = '../final/ckpt/netCRNN_80_266.pth'

#best at Cap2TxT
model_path ='./ckpt/Cap2TxT_180.pth'

#best at Cap2TxT_GoogLeNet


image_path = './Data/test/'

pred_list =[]

class_num = len(alphabet) + 1

model = Cap2TxT(hidden_size,class_num)
#model = Cap2TxT_2(64,1,class_num,hidden_size)

model = model.cuda()

model.load_state_dict(torch.load(model_path))
model.eval()
converter = utils.strLabelConverter(alphabet)
transformer=dataset.resizeNormalize((160,64))
for i in range(1000):
    image = Image.open(image_path+str(i)+'.png').convert('L')
    image = transformer(image)
    if torch.cuda.is_available():
        image = image.cuda()
    image = image.view(1, *image.size())
    image = Variable(image)
    preds = model(image)

    _, preds = preds.max(2)
    preds = preds.transpose(1, 0).contiguous().view(-1)
    preds_size = Variable(torch.LongTensor([preds.size(0)]))
    pred = converter.decode(preds.data, preds_size.data, raw=False)
    pred_list.append(pred)

answer='./Data/test.txt'
answer_list=list()
with open (answer) as f:
    for line in f:
        answer_list.append(line.rstrip('\n'))

char_correct = 0
word_correct = 0
total = 0

for i in range(1000): # size of test set is 1000
    char_count=0
    c0,c1,c2,c3,c4,c5,c6 = get_char_count(oh_encoding(pred_list[i]))
    d0,d1,d2,d3,d4,d5,d6 = get_char_count(oh_encoding(answer_list[i]))
    c = '%s%s%s%s%s%s%s' % (c0, c1, c2, c3, c4, c5, c6)
    d = '%s%s%s%s%s%s%s' % (d0, d1, d2, d3, d4, d5, d6)
    char_count += (c0==d0)+(c1==d1)+(c2==d2)+(c3==d3)+(c4==d4)+(c5==d5)+(c6==d6)
    char_correct += char_count
    if(bool(str(answer_list[i]) in str(c))):
        word_correct+=1
    total += 1

print('---char correct---')
print(100*(char_correct/(total*7)), end=' ')
print('%')
print('---word correct---')
print(100*word_correct/total, end=' ')
print('%')



---char correct---
77.55714285714286 %
---word correct---
62.6 %


In [58]:
'''Find the best weight from my experiment results.'''

def get_char_count(arg1):
    c0 = ALL_CHAR_SET[np.argmax(arg1[0:ALL_CHAR_SET_LEN])]
    c1 = ALL_CHAR_SET[np.argmax(arg1[ALL_CHAR_SET_LEN:ALL_CHAR_SET_LEN*2])]
    c2 = ALL_CHAR_SET[np.argmax(arg1[ALL_CHAR_SET_LEN*2:ALL_CHAR_SET_LEN*3])]
    c3 = ALL_CHAR_SET[np.argmax(arg1[ALL_CHAR_SET_LEN*3:ALL_CHAR_SET_LEN*4])]
    c4 = ALL_CHAR_SET[np.argmax(arg1[ALL_CHAR_SET_LEN*4:ALL_CHAR_SET_LEN*5])]
    c5 = ALL_CHAR_SET[np.argmax(arg1[ALL_CHAR_SET_LEN*5:ALL_CHAR_SET_LEN*6])]
    c6 = ALL_CHAR_SET[np.argmax(arg1[ALL_CHAR_SET_LEN*6:ALL_CHAR_SET_LEN*7])]
    return c0, c1, c2,c3, c4, c5, c6 

def oh_encoding(a):
    label_oh = []
    for i in range(7):
        if i<len(a):
            label_oh+=encode(a[i])
        else:
            label_oh+=encode('NONE')
    return label_oh



#model_path = '../final/ckpt/netCRNN_80_266.pth'

#best at Cap2TxT
#model_path ='./ckpt/Cap2TxT_180.pth'

#best at Cap2TxT_GoogLeNet
#model_path = './ckpt/Cap2TxT_final.pth'
#model_path = './ckpt/Cap2TxT_GoogLe_150.pth'
image_path = './Data/test/'
class_num = len(alphabet) + 1
converter = utils.strLabelConverter(alphabet)
transformer=dataset.resizeNormalize((160,64))

answer='./Data/test.txt'
answer_list=list()
with open (answer) as f:
    for line in f:
        answer_list.append(line.rstrip('\n'))

#Let's find best fitted model among 0~250 epoch that performs best at test set!

max_accuracy = 0
best_epoch = 0
epoch=0
while epoch <250 :
    model_path = './ckpt/Cap2TxT_GoogLe_'+str(epoch)+'.pth'
    pred_list =[]
    model = Cap2TxT(hidden_size,class_num)
    #model = Cap2TxT_2(64,1,class_num,hidden_size)
    model = model.cuda()
    model.load_state_dict(torch.load(model_path))
    model.eval()
    
    for i in range(1000):
        image = Image.open(image_path+str(i)+'.png').convert('L')
        image = transformer(image)
        if torch.cuda.is_available():
            image = image.cuda()
        image = image.view(1, *image.size())
        image = Variable(image)
        preds = model(image)

        _, preds = preds.max(2)
        preds = preds.transpose(1, 0).contiguous().view(-1)
        preds_size = Variable(torch.LongTensor([preds.size(0)]))
        pred = converter.decode(preds.data, preds_size.data, raw=False)
        pred_list.append(pred)

    char_correct = 0
    word_correct = 0
    total = 0

    for i in range(1000): # size of test set is 1000
        char_count=0
        c0,c1,c2,c3,c4,c5,c6 = get_char_count(oh_encoding(pred_list[i]))
        d0,d1,d2,d3,d4,d5,d6 = get_char_count(oh_encoding(answer_list[i]))
        c = '%s%s%s%s%s%s%s' % (c0, c1, c2, c3, c4, c5, c6)
        d = '%s%s%s%s%s%s%s' % (d0, d1, d2, d3, d4, d5, d6)
        char_count += (c0==d0)+(c1==d1)+(c2==d2)+(c3==d3)+(c4==d4)+(c5==d5)+(c6==d6)
        char_correct += char_count
        if(bool(str(answer_list[i]) in str(c))):
            word_correct+=1
        total += 1
    if max_accuracy < (100*(char_correct/(total*7))+100*word_correct/total):
        max_accuracy = (100*(char_correct/(total*7))+100*word_correct/total)
        best_epoch = epoch
    print("------------------------------------")
    print("current...")
    print(epoch, end=':')
    print(100*(char_correct/(total*7))+100*word_correct/total)
    print("best...")
    print(best_epoch, end=":")
    print(max_accuracy)
    epoch += 3



------------------------------------
current...
0:36.72857142857143
best...
0:36.72857142857143
------------------------------------
current...
3:38.2
best...
3:38.2
------------------------------------
current...
6:36.32857142857143
best...
3:38.2
------------------------------------
current...
9:101.45714285714286
best...
9:101.45714285714286
------------------------------------
current...
12:111.01428571428572
best...
12:111.01428571428572
------------------------------------
current...
15:121.2
best...
15:121.2
------------------------------------
current...
18:86.57142857142857
best...
15:121.2
------------------------------------
current...
21:147.22857142857143
best...
21:147.22857142857143
------------------------------------
current...
24:148.51428571428573
best...
24:148.51428571428573
------------------------------------
current...
27:138.38571428571427
best...
24:148.51428571428573
------------------------------------
current...
30:49.800000000000004
best...
24:148.51428571

In [59]:
print(best_epoch)

231


In [60]:
'''Test'''

def get_char_count(arg1):
    c0 = ALL_CHAR_SET[np.argmax(arg1[0:ALL_CHAR_SET_LEN])]
    c1 = ALL_CHAR_SET[np.argmax(arg1[ALL_CHAR_SET_LEN:ALL_CHAR_SET_LEN*2])]
    c2 = ALL_CHAR_SET[np.argmax(arg1[ALL_CHAR_SET_LEN*2:ALL_CHAR_SET_LEN*3])]
    c3 = ALL_CHAR_SET[np.argmax(arg1[ALL_CHAR_SET_LEN*3:ALL_CHAR_SET_LEN*4])]
    c4 = ALL_CHAR_SET[np.argmax(arg1[ALL_CHAR_SET_LEN*4:ALL_CHAR_SET_LEN*5])]
    c5 = ALL_CHAR_SET[np.argmax(arg1[ALL_CHAR_SET_LEN*5:ALL_CHAR_SET_LEN*6])]
    c6 = ALL_CHAR_SET[np.argmax(arg1[ALL_CHAR_SET_LEN*6:ALL_CHAR_SET_LEN*7])]
    return c0, c1, c2,c3, c4, c5, c6 

def oh_encoding(a):
    label_oh = []
    for i in range(7):
        if i<len(a):
            label_oh+=encode(a[i])
        else:
            label_oh+=encode('NONE')
    return label_oh

#best weight of Cap2TxT Network.
model_path ='./ckpt/Cap2TxT_GoogLe_'+str(best_epoch)+'.pth'
image_path = './Data/test/'

pred_list =[]

class_num = len(alphabet) + 1

model = Cap2TxT(hidden_size,class_num)
model = model.cuda()

model.load_state_dict(torch.load(model_path))
model.eval()
converter = utils.strLabelConverter(alphabet)
transformer=dataset.resizeNormalize((160,64))
for i in range(1000):
    image = Image.open(image_path+str(i)+'.png').convert('L')
    image = transformer(image)
    if torch.cuda.is_available():
        image = image.cuda()
    image = image.view(1, *image.size())
    image = Variable(image)
    preds = model(image)

    _, preds = preds.max(2)
    preds = preds.transpose(1, 0).contiguous().view(-1)
    preds_size = Variable(torch.LongTensor([preds.size(0)]))
    pred = converter.decode(preds.data, preds_size.data, raw=False)
    pred_list.append(pred)

answer='./Data/test.txt'
answer_list=list()
with open (answer) as f:
    for line in f:
        answer_list.append(line.rstrip('\n'))

char_correct = 0
word_correct = 0
total = 0

for i in range(1000): # size of test set is 1000
    char_count=0
    c0,c1,c2,c3,c4,c5,c6 = get_char_count(oh_encoding(pred_list[i]))
    d0,d1,d2,d3,d4,d5,d6 = get_char_count(oh_encoding(answer_list[i]))
    c = '%s%s%s%s%s%s%s' % (c0, c1, c2, c3, c4, c5, c6)
    d = '%s%s%s%s%s%s%s' % (d0, d1, d2, d3, d4, d5, d6)
    char_count += (c0==d0)+(c1==d1)+(c2==d2)+(c3==d3)+(c4==d4)+(c5==d5)+(c6==d6)
    char_correct += char_count
    if(bool(str(answer_list[i]) in str(c))):
        word_correct+=1
    total += 1

print('---char correct---')
print(100*(char_correct/(total*7)), end=' ')
print('%')
print('---word correct---')
print(100*word_correct/total, end=' ')
print('%')

---char correct---
88.2 %
---word correct---
77.9 %
